In [ ]:
bronze_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/silver/"
gold_path   = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/gold/"
resource_path = "/Volumes/<CATALOG>/<SCHEMA>/bikestore_resource/origem"
resource_path_volume = '/Volumes/aulas_databricks/azure/bikestore_resource/origem/'


In [0]:
%python
bronze_map = {
    #"tmp_bronze_brands":      f"{bronze_path}/brands/",
    #"tmp_bronze_categories":  f"{bronze_path}/categories/",
    #"tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    #"tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    #"tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%python
df_orders_silver = spark.sql("""
with order_items as(
SELECT 
   OI.order_id
  ,OI.item_id
  ,OI.product_id
  ,OI.quantity
  ,OI.list_price 
  --,(OI.list_price * OI.quantity) value_test
  ,ROUND((OI.list_price * OI.quantity)* (1-OI.discount),2) AS total_sale
  ,OI.discount
FROM tmp_bronze_order_items OI

)

-- select final ''
SELECT 
  ORD.order_id,
  ORD.customer_id,
  CASE 
      WHEN ORD.order_status = 1 THEN 'Pending'
      WHEN ORD.order_status = 2 THEN 'Processing'
      WHEN ORD.order_status = 3 THEN 'Shipped'
      WHEN ORD.order_status = 4 THEN 'Delivered'
      ELSE 'Unknown'
    END status,
  ORD.order_status,
  ORD.order_date,
  ORD.required_date,
  ORD.shipped_date,
  --ORD.store_id,
  --st.store_id AS store_id2store,
  ST.store_name,
  ST.state,
  ST.city,
  --ORD.staff_id,
  --STF.staff_id AS IDSTAF2,
  STF.first_name AS first_name_staff,
  STF.active AS active_staff,
  STF.email,
  OIT.product_id,
  OIT.quantity,
  OIT.total_sale,
  OIT.list_price,
  OIT.discount
FROM   tmp_bronze_orders ORD
LEFT JOIN tmp_bronze_stores ST  ON ORD.store_id = ST.store_id
LEFT JOIN tmp_bronze_staffs STF ON ORD.staff_id = STF.staff_id
LEFT JOIN       order_items OIT ON ORD.order_id = OIT.order_id
                              """)

# salvar em Delta na silver 
df_orders_silver.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{silver_path}/orders')